In [98]:
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)
import copy
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package genesis to
[nltk_data]     /Users/gwenythportillowightman/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gwenythportillowightman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gwenythportillowightman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gwenythportillowightman/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [99]:
# Format tags columns in df

def format_tags(df):
    tags = []
    tag_lists = []

    for subjects in df.subjects:
        if type(subjects) is str:
            s = subjects.split(",")
        else:
            if type(subjects) is list:
                s = subjects
            else:
                s = []
        s = [t.lstrip().rstrip() for t in s]
        tag_lists.append(s)
        for tag in s:
            tags.append(tag)
    df['tags'] = tag_lists
    return df, tags

In [100]:
# Select claims from relevant categories
health_tags = ['Health', 'Health News', "Health Care", 'Medical', 'Public Health']

In [101]:
# Helper function for masking dataframe with relevant tags
def health(x):
    for t in health_tags:
        if t in x:
            return True
    return False

In [102]:
# Importing the dataset
data_file_path = "/Users/gwenythportillowightman/OneDrive - Johns Hopkins/fall-2022/interpretable_ml_design/PUBHEALTH/train.tsv"          

dataset = pd.read_csv(data_file_path, sep='\t')
dataset, dataset_tags = format_tags(dataset)

mask = dataset['tags'].apply(lambda x: health(x))
dataset = dataset[mask]

# text_col contains the column name of where claims are found
# answer_col contains the column name of where post labels (true, false, etc.) are found
text_col = "text"
answer_col = "label"

# Rename the claim column to "text" and label column to "label_categorical"
dataset.rename(columns = {"claim": "text", "label":"label_categorical"}, inplace = True)
# Make the categorical labels into numbers (0, 1, 2, 3)
dataset["label"] = pd.factorize(dataset["label_categorical"])[0]
dataset = dataset.dropna(subset=[text_col])
dataset.reset_index(drop=True, inplace=True)

# Make a copy of the 'text' column
dataset['text_original'] = dataset['text']

print("\nSize of input file is ", dataset.shape)


Size of input file is  (3596, 12)


In [103]:
import re
nltk.download('stopwords')
s = stopwords.words('english')
#add additional stop words
s.extend(['today', 'tomorrow', 'outside', 'out', 'there'])
ps = nltk.wordnet.WordNetLemmatizer()
for i in range(dataset.shape[0]):
    review = dataset.loc[i,'text']
    review = re.sub('[^a-zA-Z]', ' ', dataset.loc[i,'text'])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in s]
    review = ' '.join(review)
    dataset.loc[i, 'text'] = review
    X_train = dataset['text']
y_train = dataset['label']
print("Below is the sample of training text after removing the stop words")
print(dataset['text'][:10])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gwenythportillowightman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Below is the sample of training text after removing the stop words
0    britain reveal trial criterion coronavirus ant...
1    u say result encouraging healthcare delivery r...
2    latest trial j j talc litigation get way calif...
3       democrat hoping flip house trash talking trump
4        sex tech woman led startup pop ce gadget show
5                             waxed apple cause cancer
6    rhode island become second state mandate vacci...
7    brazil city lurch lockdown amid virus crisis r...
8    slovakia new government sharply ramp coronavir...
9                       coronavirus simply common cold
Name: text, dtype: object


In [104]:
class KNN_NLC_Classifer():
    def __init__(self, k=3, distance_type = 'path'):
        self.k = k
        self.distance_type = distance_type

    # This function is used for training
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    # This function runs the K(n) nearest neighbour algorithm and
    # returns the label with closest match. 
    # Predict returns the n similar sentences as a list of tuples [(sentence, score), (sentence, score), ...]
    # Takes in only one input at a time
    def predict(self, x_test):
        test_corpus = []
        
        # Preprocess the full x_test input
        x_test_copy = copy.deepcopy(x_test)
        x_test_copy = re.sub('[^a-zA-Z]', ' ', x_test_copy)
        x_test_copy = x_test_copy.lower()
        x_test_copy = x_test_copy.split()
        x_test_copy = [ps.lemmatize(word) for word in x_test_copy if not word in s]
        x_test_copy = ' '.join(x_test_copy)
        
        # Preprocess sentences of the input
        sentences = sent_tokenize(x_test)
        
        for i in range(len(sentences)):
            review = re.sub('[^a-zA-Z]', ' ', sentences[i])
            review = review.lower()
            review = review.split()
            review = [ps.lemmatize(word) for word in review if not word in s]
            review = ' '.join(review)
            test_corpus.append(review)
            
        test_corpus.append(x_test_copy)
            
        self.x_test = test_corpus
    
        # {score: [(index of sentence in `test_corpus`, similar sentence index in `dataset`)], ...}
        all_top_scores_dict = {}

        # Iterate over sentences of the input
        for i in range(len(self.x_test)):
            print(self.x_test[i])
            
            # {score: similar_sentence_index_in_`dataset`, ...}
            score_to_index_dict = {}
            
            # Iterate over training examples and find sentence similarity scores
            for j in range(self.x_train.shape[0]): 
                score = self.document_similarity(self.x_test[i], self.x_train[j])
                score_to_index_dict[score] = j

            sorted_scores = list(score_to_index_dict.keys())
            sorted_scores.sort(reverse=True)

            # Get the top k similar sentences for the current sentence (x_test[i])
            for k in range(self.k):
                score = sorted_scores[k]
                
                if score in all_top_scores_dict:
                    all_top_scores_dict[score].append( (i, score_to_index_dict[score]) )
                else:
                    all_top_scores_dict[score] = [ (i, score_to_index_dict[score]) ]
                    
        # Get the top k scoring sentences and similar sentences from all_top_scores_dict
        sorted_scores = list(all_top_scores_dict.keys())
        sorted_scores.sort(reverse=True)
        
        # [ ((index_of_sentence_in_input, index_of_similar_sentence_in_`dataset`), score), ...]
        similar_texts_list = []
        
        for k in range(self.k):
            score = sorted_scores[k]
            new_tuple = (all_top_scores_dict[score], score)
            similar_texts_list.append(new_tuple)

        return similar_texts_list
    
    def convert_tag(self, tag):
        """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
        tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
        try:
            return tag_dict[tag[0]]
        except KeyError:
            return None

    def doc_to_synsets(self, doc):
        """
            Returns a list of synsets in document.
            Tokenizes and tags the words in the document doc.
            Then finds the first synset for each word/tag combination.
        If a synset is not found for that combination it is skipped.

        Args:
            doc: string to be converted

        Returns:
            list of synsets
        """
        tokens = word_tokenize(doc+' ')
        
        l = []
        tags = nltk.pos_tag([tokens[0] + ' ']) if len(tokens) == 1 else nltk.pos_tag(tokens)
        
        for token, tag in zip(tokens, tags):
            syntag = self.convert_tag(tag[1])
            syns = wn.synsets(token, syntag)
            if (len(syns) > 0):
                l.append(syns[0])
        return l  
    
    def similarity_score(self, s1, s2, distance_type = 'path'):
        """
        Calculate the normalized similarity score of s1 onto s2
        For each synset in s1, finds the synset in s2 with the largest similarity value.
        Sum of all of the largest similarity values and normalize this value by dividing it by the
        number of largest similarity values found.

        Args:
          s1, s2: list of synsets from doc_to_synsets

        Returns:
          normalized similarity score of s1 onto s2
        """
        s1_largest_scores = []

        for i, s1_synset in enumerate(s1, 0):
            max_score = 0
            for s2_synset in s2:
                if distance_type == 'path':
                    score = s1_synset.path_similarity(s2_synset, simulate_root = False)
                else:
                    score = s1_synset.wup_similarity(s2_synset)                  
                if score != None:
                    if score > max_score:
                        max_score = score

            if max_score != 0:
                s1_largest_scores.append(max_score)

        mean_score = np.mean(s1_largest_scores)

        return mean_score 
    
    def document_similarity(self,doc1, doc2):
        """Finds the symmetrical similarity between doc1 and doc2"""

        synsets1 = self.doc_to_synsets(doc1)
        synsets2 = self.doc_to_synsets(doc2)
          
        return (self.similarity_score(synsets1, synsets2) + self.similarity_score(synsets2, synsets1)) / 2


In [ ]:
# 4. Train the Classifier
classifier = KNN_NLC_Classifer(k=3, distance_type='path')
classifier.fit(X_train, y_train)

final_test_list = 'They are coated in toxic chemicals. Dryer sheets are one of the very worst things from a chemical allergy standpoint.'

# test_corpus = []
# for i in range(len(final_test_list)):
#     review = re.sub('[^a-zA-Z]', ' ', final_test_list[i])
#     review = review.lower()
#     review = review.split()

#     review = [ps.lemmatize(word) for word in review if not word in s]
    
#     review = ' '.join(review)
#     test_corpus.append(review)

# print("Predicting...")
y_pred_final = classifier.predict(final_test_list)
print(y_pred_final)
print()
print("Top 3 similar examples:")

for i, result in enumerate (y_pred_final):
#     print(f"index of sentence in input & similar sentence index in dataset: {result[0]}, score: {result[1]}")
    original_sentence_index = result[0][0][0]
    similar_sentence_index = result[0][0][1]
    similar_sentence_data = dataset.iloc[[similar_sentence_index]].values.tolist()[0]
    text_original_column_index = 11
    label_categorical_column_index = 7
    print(f'Original sentence: {similar_sentence_data[text_original_column_index]}')
    print(f'Label: {similar_sentence_data[label_categorical_column_index]}')
    print()

coated toxic chemical
dryer sheet one worst thing chemical allergy standpoint
coated toxic chemical dryer sheet one worst thing chemical allergy standpoint
